# I - Importations

In [1]:
import os
import numpy as np
from helpers.labels import labels
import tensorflow as tf
from tensorflow.keras import models # type: ignore
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, concatenate, Input, MaxPooling2D # type: ignore
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras.utils import Sequence # type: ignore
from tensorflow.keras.preprocessing.image import load_img, img_to_array # type: ignore
from tensorflow.keras.applications import VGG16 # type: ignore
import albumentations as Album

import mlflow
import mlflow.tensorflow

# II - Paramétrages

In [2]:
TRAIN_IMAGE_DIR = "datas/train/image"
TRAIN_MASK_DIR = "datas/train/labelIds"
VAL_IMAGE_DIR = "datas/val/image"
VAL_MASK_DIR = "datas/val/labelIds"
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32
LIMIT_NUM_IMAGES = None

mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")
mlflow.set_experiment("experiment")
mlflow.tensorflow.autolog()

2024/10/28 22:04:17 WARNING mlflow.utils.autologging_utils: MLflow tensorflow autologging is known to be compatible with 2.7.4 <= tensorflow <= 2.17.0, but the installed version is 2.18.0. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a compatible version, or try upgrading MLflow.


# III - Classes

In [3]:
class ImageMaskGenerator(Sequence):

    
    def __init__(self, image_dir, mask_dir, class_mapping, batch_size=32, img_height=1024, img_width=2048, shuffle=True, limit_num_images=None, augment=False, **kwargs):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.class_mapping = class_mapping
        self.batch_size = batch_size
        self.img_height = img_height
        self.img_width = img_width
        self.limit_num_images = limit_num_images
        self.augment = augment
        self.shuffle = shuffle
        self.image_mask_pairs = self.create_image_mask_pairs()
        self.indexes = np.arange(len(self.image_mask_pairs))

        self.augmentation_pipeline = Album.Compose([
            Album.HorizontalFlip(p=0.5),
            Album.RandomBrightnessContrast(p=0.2),
            Album.Rotate(limit=35, p=0.5)
        ])

        self.on_epoch_end()

        super().__init__(**kwargs)

    def __len__(self):
        return int(np.ceil(len(self.image_mask_pairs) / self.batch_size))

    def __getitem__(self, idx):
        batch_indexes = self.indexes[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_images, batch_masks = self.__data_generation(batch_indexes)
        return batch_images, batch_masks

    def __data_generation(self, batch_indexes):

        images = []
        masks = []
        batch_image_mask_pairs = [self.image_mask_pairs[i] for i in batch_indexes]
        for image_name, mask_name in batch_image_mask_pairs:
            image_path = os.path.join(self.image_dir, image_name)
            mask_path = os.path.join(self.mask_dir, mask_name)

            img = img_to_array(load_img(image_path, target_size=(self.img_height, self.img_width)), dtype=np.float32)

            mask = img_to_array(load_img(mask_path, target_size=(self.img_height, self.img_width)), dtype=np.float32)
            mask = mask[..., 0]

            if self.augment:
                augmented = self.augmentation_pipeline(image=img, mask=mask)
                img, mask = augmented["image"], augmented["mask"]

            mapped_mask = np.zeros((self.img_height, self.img_width, 8), dtype=np.int8)
            for val, mapped_val in self.class_mapping.items():
                mapped_mask[mask == val, mapped_val] = 1

            images.append(img / 255.0)
            masks.append(mapped_mask)

        return np.array(images, dtype=np.float32), np.array(masks, dtype=np.int8)

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def create_image_mask_pairs(self):

        if self.limit_num_images:
            image_files = [f for i, f in enumerate(os.listdir(self.image_dir)) if i < self.limit_num_images]
            mask_files = [f for i, f in enumerate(os.listdir(self.mask_dir)) if i < self.limit_num_images]
        else:
            image_files = [f for f in os.listdir(self.image_dir)]
            mask_files = [f for f in os.listdir(self.mask_dir)]

        return list(zip(image_files, mask_files))

    

# IV - Fonctions

In [4]:
def unet_mini(input_shape):
    inputs = Input(shape=input_shape)

    # Contracting path
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(128, (3, 3), activation='relu', padding='same')(c3)

    # Expanding path
    u4 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c3)
    u4 = concatenate([u4, c2])
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(u4)
    c4 = Conv2D(64, (3, 3), activation='relu', padding='same')(c4)

    u5 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c4)
    u5 = concatenate([u5, c1])
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(u5)
    c5 = Conv2D(32, (3, 3), activation='relu', padding='same')(c5)

    outputs = Conv2D(8, (1, 1), activation='softmax')(c5)  # 8 classes

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model


In [5]:
def unet_vgg16(input_shape=(128, 128, 3), num_classes=8):
    # Encoder: VGG16 backbone (pre-trained on ImageNet)
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze VGG16 layers
    
    # Extract specific feature layers for skip connections in the U-Net architecture
    f1 = base_model.get_layer("block1_pool").output  # (64x64)
    f2 = base_model.get_layer("block2_pool").output  # (32x32)
    f3 = base_model.get_layer("block3_pool").output  # (16x16)
    f4 = base_model.get_layer("block4_pool").output  # (8x8)
    f5 = base_model.get_layer("block5_pool").output  # (4x4)
    
    # Decoder: U-Net style upsampling with skip connections
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(f5)
    up6 = concatenate([up6, f4])  # Skip connection
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)
    
    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = concatenate([up7, f3])  # Skip connection
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)
    
    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = concatenate([up8, f2])  # Skip connection
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)
    
    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = concatenate([up9, f1])  # Skip connection
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)
    
    # Final upsampling layer
    up10 = UpSampling2D((2, 2))(conv9)
    output = Conv2D(num_classes, (1, 1), activation='softmax')(up10)
    
    # Model
    model = Model(inputs=base_model.input, outputs=output)
    
    return model

In [6]:
def dice_loss(y_true, y_pred, smooth=1):
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return 1 - ((2. * intersection + smooth) /
                (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth))

In [7]:
def total_loss(y_true, y_pred):
    # Combinaison de Dice Loss et Binary Cross-Entropy
    dice = dice_loss(y_true, y_pred)
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    return dice + bce

In [8]:
import tensorflow as tf

def iou_multi_class(y_true, y_pred, num_classes=8):
    # Convertir les prédictions en classes
    y_pred = tf.argmax(y_pred, axis=-1)  # Prendre l'indice de la classe avec la probabilité maximale
    y_pred = tf.expand_dims(y_pred, axis=-1)  # Ajouter une dimension pour le batch

    # Calculer l'IoU pour chaque classe
    iou_list = []
    for i in range(num_classes):
        intersection = tf.reduce_sum(tf.cast((y_true == i) & (y_pred == i), tf.float32))
        union = tf.reduce_sum(tf.cast(y_true == i, tf.float32)) + tf.reduce_sum(tf.cast(y_pred == i, tf.float32)) - intersection
        iou = intersection / (union + tf.keras.backend.epsilon())
        iou_list.append(iou)

    return tf.reduce_mean(iou_list)  # Retourner la moyenne des IoUs


In [9]:
def dice_coefficient_multi_class(y_true, y_pred, num_classes=8):
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.expand_dims(y_pred, axis=-1)

    dice_list = []
    for i in range(num_classes):
        intersection = tf.reduce_sum(tf.cast((y_true == i) & (y_pred == i), tf.float32))
        dice = (2. * intersection + tf.keras.backend.epsilon()) / (tf.reduce_sum(tf.cast(y_true == i, tf.float32)) + tf.reduce_sum(tf.cast(y_pred == i, tf.float32)) + tf.keras.backend.epsilon())
        dice_list.append(dice)

    return tf.reduce_mean(dice_list)  # Retourner la moyenne des Dice coefficients


# IV - Initialisations

In [10]:
train_image_list = os.listdir(TRAIN_IMAGE_DIR)
train_mask_list = os.listdir(TRAIN_MASK_DIR)
val_image_list = os.listdir(VAL_IMAGE_DIR)
val_mask_list = os.listdir(VAL_MASK_DIR)
id2catid = {label.id : label.categoryId for label in labels}

# Code

In [11]:
epochs = 5
params_loss = [{"name":"categorical_crossentropy", "loss":"categorical_crossentropy"}, {"name":"dice_loss", "loss":dice_loss},{"name":"total_loss", "loss":total_loss}]

for param in params_loss:
    name_run = f"unet_mini with {param["name"]}"
    runs = mlflow.search_runs(filter_string=f"tags.mlflow.runName='{name_run}'").index
    if len(runs) == 0:
        train_sequence = ImageMaskGenerator(TRAIN_IMAGE_DIR, TRAIN_MASK_DIR, id2catid, batch_size=BATCH_SIZE, img_height=IMG_HEIGHT, img_width=IMG_WIDTH, limit_num_images=LIMIT_NUM_IMAGES)
        val_sequence = ImageMaskGenerator(VAL_IMAGE_DIR, VAL_MASK_DIR, id2catid, batch_size=BATCH_SIZE, img_height=IMG_HEIGHT, img_width=IMG_WIDTH, shuffle=False, limit_num_images=LIMIT_NUM_IMAGES)
        model_mini = unet_mini((IMG_HEIGHT, IMG_WIDTH, 3))
        model_mini.compile(optimizer="adam", loss=param["loss"], metrics=["accuracy", iou_multi_class, dice_coefficient_multi_class])
        with mlflow.start_run(run_name=name_run):
            model_mini.fit(train_sequence, validation_data=val_sequence, epochs=epochs, batch_size=BATCH_SIZE)

for param in params_loss:
    name_run = f"unet_vgg16 with {param["name"]}"
    runs = mlflow.search_runs(filter_string=f"tags.mlflow.runName='{name_run}'").index
    if len(runs) == 0:
        train_sequence = ImageMaskGenerator(TRAIN_IMAGE_DIR, TRAIN_MASK_DIR, id2catid, batch_size=BATCH_SIZE, img_height=IMG_HEIGHT, img_width=IMG_WIDTH, limit_num_images=LIMIT_NUM_IMAGES)
        val_sequence = ImageMaskGenerator(VAL_IMAGE_DIR, VAL_MASK_DIR, id2catid, batch_size=BATCH_SIZE, img_height=IMG_HEIGHT, img_width=IMG_WIDTH, shuffle=False, limit_num_images=LIMIT_NUM_IMAGES)
        model_vgg16 = unet_vgg16((IMG_HEIGHT, IMG_WIDTH, 3))
        model_vgg16.compile(optimizer="adam", loss=param["loss"], metrics=["accuracy", iou_multi_class, dice_coefficient_multi_class])
        with mlflow.start_run(run_name=name_run):
            model_vgg16.fit(train_sequence, validation_data=val_sequence, epochs=epochs, batch_size=BATCH_SIZE)

for param in params_loss:
    name_run = f"unet_mini Augmented with {param["name"]}"
    runs = mlflow.search_runs(filter_string=f"tags.mlflow.runName='{name_run}'").index
    if len(runs) == 0:
        train_sequence = ImageMaskGenerator(TRAIN_IMAGE_DIR, TRAIN_MASK_DIR, id2catid, batch_size=BATCH_SIZE, img_height=IMG_HEIGHT, img_width=IMG_WIDTH, limit_num_images=LIMIT_NUM_IMAGES, augment=True)
        val_sequence = ImageMaskGenerator(VAL_IMAGE_DIR, VAL_MASK_DIR, id2catid, batch_size=BATCH_SIZE, img_height=IMG_HEIGHT, img_width=IMG_WIDTH, shuffle=False, limit_num_images=LIMIT_NUM_IMAGES)
        model_mini = unet_mini((IMG_HEIGHT, IMG_WIDTH, 3))
        model_mini.compile(optimizer="adam", loss=param["loss"], metrics=["accuracy", iou_multi_class, dice_coefficient_multi_class])
        with mlflow.start_run(run_name=name_run):
            model_mini.fit(train_sequence, validation_data=val_sequence, epochs=epochs, batch_size=BATCH_SIZE)

for param in params_loss:
    name_run = f"unet_vgg16 Augmented with {param["name"]}"
    runs = mlflow.search_runs(filter_string=f"tags.mlflow.runName='{name_run}'").index
    if len(runs) == 0:
        train_sequence = ImageMaskGenerator(TRAIN_IMAGE_DIR, TRAIN_MASK_DIR, id2catid, batch_size=BATCH_SIZE, img_height=IMG_HEIGHT, img_width=IMG_WIDTH, limit_num_images=LIMIT_NUM_IMAGES, augment=True)
        val_sequence = ImageMaskGenerator(VAL_IMAGE_DIR, VAL_MASK_DIR, id2catid, batch_size=BATCH_SIZE, img_height=IMG_HEIGHT, img_width=IMG_WIDTH, shuffle=False, limit_num_images=LIMIT_NUM_IMAGES)
        model_vgg16 = unet_vgg16((IMG_HEIGHT, IMG_WIDTH, 3))
        model_vgg16.compile(optimizer="adam", loss=param["loss"], metrics=["accuracy", iou_multi_class, dice_coefficient_multi_class])
        with mlflow.start_run(run_name=name_run):
            model_vgg16.fit(train_sequence, validation_data=val_sequence, epochs=epochs, batch_size=BATCH_SIZE)

2024/10/28 22:04:18 WARNING mlflow.tensorflow: Unrecognized dataset type <class '__main__.ImageMaskGenerator'>. Dataset logging skipped.
2024/10/28 22:04:18 WARNING mlflow.tensorflow: Unrecognized dataset type <class '__main__.ImageMaskGenerator'>. Dataset logging skipped.


Epoch 1/5


d:\Formations\Github\Projet_08\.venv\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(


93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3965 - dice_coefficient_multi_class: 0.5180 - iou_multi_class: 0.0837 - loss: 1.6587

93/93 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.3977 - dice_coefficient_multi_class: 0.5176 - iou_multi_class: 0.0837 - loss: 1.6556 - val_accuracy: 0.6561 - val_dice_coefficient_multi_class: 0.3412 - val_iou_multi_class: 0.0630 - val_loss: 0.9822
Epoch 2/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6903 - dice_coefficient_multi_class: 0.3430 - iou_multi_class: 0.0641 - loss: 0.8937

...

93/93 ━━━━━━━━━━━━━━━━━━━━ 212s 2s/step - accuracy: 0.7431 - dice_coefficient_multi_class: 0.3395 - iou_multi_class: 0.0617 - loss: 0.4512 - val_accuracy: 0.8012 - val_dice_coefficient_multi_class: 0.3322 - val_iou_multi_class: 0.0607 - val_loss: 0.3475
Epoch 3/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7630 - dice_coefficient_multi_class: 0.3361 - iou_multi_class: 0.0615 - loss: 0.4093

93/93 ━━━━━━━━━━━━━━━━━━━━ 212s 2s/step - accuracy: 0.7631 - dice_coefficient_multi_class: 0.3361 - iou_multi_class: 0.0615 - loss: 0.4091 - val_accuracy: 0.8157 - val_dice_coefficient_multi_class: 0.2843 - val_iou_multi_class: 0.0604 - val_loss: 0.3222
Epoch 4/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7782 - dice_coefficient_multi_class: 0.3255 - iou_multi_class: 0.0614 - loss: 0.3824

93/93 ━━━━━━━━━━━━━━━━━━━━ 211s 2s/step - accuracy: 0.7782 - dice_coefficient_multi_class: 0.3254 - iou_multi_class: 0.0614 - loss: 0.3823 - val_accuracy: 0.8210 - val_dice_coefficient_multi_class: 0.2997 - val_iou_multi_class: 0.0597 - val_loss: 0.3123
Epoch 5/5
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7878 - dice_coefficient_multi_class: 0.2893 - iou_multi_class: 0.0607 - loss: 0.3659

93/93 ━━━━━━━━━━━━━━━━━━━━ 212s 2s/step - accuracy: 0.7878 - dice_coefficient_multi_class: 0.2891 - iou_multi_class: 0.0607 - loss: 0.3660 - val_accuracy: 0.8291 - val_dice_coefficient_multi_class: 0.2310 - val_iou_multi_class: 0.0610 - val_loss: 0.3001


2024/10/29 09:18:04 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: unsupported operand type(s) for *: 'slice' and 'int'
2024/10/29 09:18:04 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/10/29 09:18:08 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\nicyu\AppData\Local\Temp\tmp2x5a5dn2\model, flavor: tensorflow). Fall back to return ['tensorflow==2.18.0', 'cloudpickle==3.1.0']. Set logging level to DEBUG to see the full traceback. 
2024/10/29 09:18:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/29 09:18:10 INFO mlflow.